In [1]:
from utils import *
device = "cuda" if torch.cuda.is_available() else "cpu"

C:\Users\dab4s\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def iterate(dataset):
    while True:
        for batch in dataset:
            yield batch

In [3]:
def trainModel(model, loaders, config):
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learningRate)
    try:
        criterion = torch.nn.BCELoss()

        testIter = iterate(loaders["val"])

        client = Client("127.0.0.1", 12945)

        for epoch in range(config.epochs):
            progress = 0
            for inputs, targets in loaders["train"]:
                model.train()
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets.float())
                loss.backward()
                optimizer.step()

                trainLoss = loss.item()

                inputs1, targets1 = next(testIter)
                with torch.no_grad():
                    model.eval()
                    outputs1 = model(inputs1)
                    loss1 = criterion(outputs1.squeeze(), targets1.float())

                    testLoss = loss1.item()

                client.send("Train Loss", trainLoss)
                client.send("Test Loss", testLoss)

                progress += 1
                print(f"\r{epoch + 1} | {progress}/{len(loaders['train'])} | Train Loss: {trainLoss} | Test Loss: {testLoss}", end="")

    except KeyboardInterrupt:
        pass
    return model, train, test, optimizer

In [ ]:
models = [PCAFormer, ResNet50, SwinTransformerV2Tiny]

dataDir = download_dataset()

config = Config().load(os.path.join("configs", "config.json"))
config.dataset.dataDir = dataDir
loaders = get_dataloaders(config)

for modelClass in models:
    model = modelClass(config.model)

    model, train, test, optimizer = trainModel(model, loaders, config)